In [91]:
from __future__ import division 
# This tells matplotlib not to try opening a new window for each plot.
#%matplotlib inline

# General libraries.
import os
import codecs
import json
import csv

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import glob
import pickle
import time
# SK-learn libraries for learning.
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import NMF

from nltk.stem import WordNetLemmatizer
import nltk

In [4]:
data=pickle.load(open("/Users/meaneylab/Box Sync/266/FinalProject/Fulldata_wY", 'rb'))
data=data.drop(["Unnamed: 0"], axis=1)

In [81]:
# load previously saved lemmatized text, see " Lemmatizing Texts.ipynb "
lemmatized_text=pickle.load(open("/Users/meaneylab/Box Sync/266/FinalProject/Text_lemmatized", "rb"))

In [88]:
lemmatized_text[0][0:1000]

u"TEXT : Check the appropriate box below if the Form 8-K file be intend to simultaneously satisfy the file obligation of the registrant under any of the follow provision ( General Instruction A.2 . below ) : see Item 2.02 Results of Operations and Financial Condition On December 17 , 2009 , Accenture issue a press release announce financial result for its first quarter of fiscal year 2010 , which fiscal quarter end on November 30 , 2009 . A copy of the press release be attach hereto as Exhibit 99.1 . All information in the press release be furnish but not file . Non-GAAP Financial Information In the attach press release Accenture disclose the follow non-GAAP financial measure : Reconciliations of these non-GAAP financial measure to the most directly comparable financial measure calculate and present in accordance with GAAP be include in the press release . While Accenture 's management believe that this non-GAAP financial information be useful in evaluate Accenture 's operations , this

In [89]:
vectorizer1=CountVectorizer(stop_words='english', min_df=10)
text_vector=vectorizer1.fit_transform(lemmatized_text)
text_vector.shape

(37503, 33530)

In [90]:
pickle.dump(text_vector, open("/Users/meaneylab/Box Sync/266/FinalProject/Text_vector_lemmatized", 'wb'))

In [92]:
##  select top features using feature selection packages
ktop=SelectKBest(chi2, k=3000).fit_transform(text_vector, labels)
ktop.shape

(37503, 3000)

In [94]:
## Non-negative factorization of the top unigram features, with 100 dimensions
start=time.time()
model100 = NMF(n_components=100, init='random', random_state=1, alpha=.1, l1_ratio=.5)
topVec100 = model100.fit_transform(ktop)
print("Finished in {:3.1f} seconds".format(time.time()-start))


Finished in 364.7 seconds


In [95]:
pickle.dump(topVec100,open("/Users/meaneylab/Box Sync/266/FinalProject/TopVec100_lemmatized", 'wb'))

### Read in previously-saved results of NMF text vectors


In [8]:
topVec100=pickle.load(open("/Users/meaneylab/Box Sync/266/FinalProject/TopVec100_lemmatized", "rb"))

In [109]:
alldata=pd.DataFrame(np.hstack((data.as_matrix(), topVec100)))
alldata.columns=np.array(['Company', 'ticker', 'Surprise', 'Reported_EPS', 'Consensus_EPS',
       'Date', 'timestamp', 'bow', 'items', 'text', 'orig_file',
       'release_time_type', 'return', 'stock_performance',
       'market_performance', 'normalized_performance', 'label']+range(100))
allfeatures=alldata.drop(["Company", "ticker",'bow', 'orig_file', 'stock_performance', \
                              'market_performance', 'normalized_performance', 'text',\
                          'timestamp' , 'Reported_EPS', 'Consensus_EPS', "items", "return"], axis=1).dropna(axis=0, how="any")
allfeatures.head()

,Surprise,Date,release_time_type,label,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,3.08,2009-12-17 00:00:00,3,DOWN,0,0,0,0.0872788,0.00243483,0,...,0.000390139,0.00306087,0.0350284,0.587139,0.00585252,0,0.0229607,0,0.0183246,0
1,25,2009-12-17 00:00:00,1,UP,0,0,0,2.876,0,0,...,0.00106202,0.00218452,0.0945596,0.283774,0.0021018,0.0881531,0,0.334748,0.0215783,0
2,2.38,2009-12-17 00:00:00,3,STAY,0.192988,0,0,0,0.00152716,0,...,0,0,0.0371115,0.392072,0.0324689,0.0151895,0,0.137712,0,0
3,3.77,2009-12-17 00:00:00,1,DOWN,0.205799,0,0,0,0,0,...,0,0.0129627,0.020562,0,0,0,0,0.138232,0.0270192,0
4,6.94,2009-12-17 00:00:00,1,UP,0.195125,0,0,0.358698,0,0,...,0.00254715,0,0.00669368,1.24026,0.0141996,0.00149622,0,0.262501,0,0


In [132]:
train_data = allfeatures.loc[allfeatures.Date < pd.to_datetime('2009-01-01'), :].drop(['Date', 'label'], axis=1)
dev_data = allfeatures.loc[(allfeatures.Date >= pd.to_datetime('2009-01-01')) & \
                           (allfeatures.Date <= pd.to_datetime('2010-12-31')), :].drop(['Date', 'label'], axis=1)
test_data = allfeatures.loc[allfeatures.Date >= pd.to_datetime('2011-01-01'), :].drop(['Date'], axis=1)
test_data=test_data[test_data['release_time_type']!=pd.Timestamp('2012-08-07 09:30:00')]
test_label=test_data['label']
test_data=test_data.drop(["label"], axis=1)

In [133]:
train_label=allfeatures.loc[allfeatures.Date < pd.to_datetime('2009-01-01'), "label"]
dev_label = allfeatures.loc[(allfeatures.Date >= pd.to_datetime('2009-01-01')) & \
                           (allfeatures.Date <= pd.to_datetime('2010-12-31')), 'label']


In [134]:
print train_data.shape, dev_data.shape, test_data.shape

(17449, 102) (9715, 102) (10191, 102)


In [27]:
rf=RandomForestClassifier(n_estimators=2000)
model=rf.fit(train_data, train_label)

In [135]:
#pickle.dump(model, open("/Users/meaneylab/Box Sync/266/FinalProject/RF_unlemmatized_2000Tree", 'wb'))

In [136]:
model_lem=rf.fit(train_data, train_label)

In [137]:
#pickle.dump(model, open("/Users/meaneylab/Box Sync/266/FinalProject/RF_lemmatized_2000Tree", 'wb'))

In [141]:
# Dev set accuracy
preds_dev = model_lem.predict(dev_data)
F_Score_dev = metrics.f1_score(dev_label, preds, average='weighted')
pred_probas_dev = model_lem.predict_proba(dev_data)
#model_output(pred_probas, F_Score, preds)
conf_dev=confusion_matrix(dev_label.values, preds_dev,labels=["UP", "STAY", "DOWN"] , )
print(conf_dev/len(preds_dev))
print("F-score : {:3.3f}".format(F_Score_dev))
print("Accuracy : {:3.3f}".format(np.sum(preds_dev==dev_label)/len(dev_label)))

[[ 0.299228    0.01791045  0.05898096]
 [ 0.16222337  0.04632012  0.08378796]
 [ 0.13772517  0.02367473  0.17014925]]
F-score : 0.481
Accuracy : 0.516


In [148]:
# Test set accuracy
preds_test = model.predict(test_data)
F_Score_test = metrics.f1_score(test_label, preds_test, average='weighted')
pred_probas = model.predict_proba(test_data)
#model_output(pred_probas, F_Score, preds)
conf_test=confusion_matrix(test_label, preds_test)
print(conf_test/len(preds_test))
print("F-score : {:3.3f}".format(F_Score_test))
print("Accuracy : {:3.3f}".format(np.sum(preds_test==test_label)/len(test_label)))

[[ 0.1627907   0.06083799  0.10028456]
 [ 0.08419193  0.10695712  0.13757237]
 [ 0.04631538  0.0456285   0.25542145]]
F-score : 0.512
Accuracy : 0.525


In [133]:
#Method for printing out Roc curve
from sklearn.metrics import roc_curve, auc

def model_output(pred_probas, F_Score, preds, title=''):
    print (("Number of positive prediction: %d") % (preds.sum()))
    print ("Model F-Score = %0.4f "%(F_Score))  #Sum up Squared Weights
    accuracy = np.where(preds==dev_labels, 1, 0).sum() / float(len(dev_labels))
    print ("Accuracy = %0.4f"    % (accuracy))
    print ('precision_score: %f'  % metrics.precision_score(dev_labels, preds))
    print ('recall_score: %0.4f' % metrics.recall_score(dev_labels, preds))
    #print ('roc_auc_score: %0.4f'% metrics.roc_auc_score(dev_labels, preds, 'weighted'))
    
    # to reproduce from Kaggle, dont change weighted argument, just use default
    # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    # https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/auc.py
    
    # don't use 0 or 1, instead use the probabilities but think about it as a ranking score
    # with auc, just want positive examples ranked over the negative examples,
    # does not matter if .99 > .90 or .11 vs. .01
    print ('roc_auc_score: %0.4f'% metrics.roc_auc_score(dev_labels, pred_probas[:,1]))
    
    # if just printing, use roc_auc_score as above, computes the roc all at once
    # if plotting, use roc_auc as below, computes one step at a time
    # returns the scores at each threshold
    # https://hsto.org/files/54b/611/188/54b611188a8b4b2a9ca1e41884f21a3f.png
    
    # add thresholds
    
    fpr,tpr,thresholds = roc_curve(dev_labels, pred_probas[:,1])
    table_of_thresholds = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds})
    roc_auc = auc(fpr,tpr)
    plt.plot(fpr,tpr,label='area = %.4f' %roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.title(title)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    #return table_of_thresholds